In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 
img_width, img_height = 128, 128

input_shape = (img_width, img_height, 3) ## we checked that the RGB channel comes last in the data format of these images

epochs = 10
batch_size = 16

In [2]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy']) ## try accuracy for now, can change later

In [3]:
## clearly there were some problems with the last iteration of the model since accuracy started at 75% and remained the same....
## this could possibly be a problem with class imbalancing, so let's try using class weights

datagen = ImageDataGenerator(rescale = 1/255) ## resizing pixels ?
 
train_data = datagen.flow_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\_FINAL DATASETS\alzheimer mri preprocessed dataset\train_test_val split\train',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')
 
validation_data = datagen.flow_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\_FINAL DATASETS\alzheimer mri preprocessed dataset\train_test_val split\val',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')
 
model.fit(
    train_data,
    steps_per_epoch = 4479 // batch_size,
    epochs=epochs,
    validation_data = validation_data,
    validation_steps = 642 // batch_size,
    class_weight = {0: 4479/627, 1: 4479/44, 2: 4479/2240, 3: 4479/1568}) ## these class weights are based on the train dataset (total train samples/train samples for class i)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\lucia\\OneDrive - University of Calgary\\University\\Year 3\\Semester 2\\Bmen 415\\Coursework\\Course Project\\_FINAL DATASETS\\alzheimer mri preprocessed dataset\\train_test_val split\\train'

In [ ]:
model.save('CNN_model.h5')

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np
 
from keras.models import load_model
 
model = load_model('CNN_model.h5')
 
## let's test one of the mild demented images reserved in the training dataset
image = load_img(r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\_FINAL DATASETS\alzheimer mri preprocessed dataset\train_test_val split\test\Mild_Demented\mild_55.jpg', target_size = (128, 128))
img = np.array(image) / 255
img = img.reshape(1, 128, 128, 3)

prediction = model.predict(img)
print(prediction)
print("Predicted Class: ", prediction[0][0])

[[0.25720322]]
Predicted Class:  0.25720322


In [ ]:
## two issues visible here... accuracy is still at 0.75 constantly (so it is NOT a class imbalance problem?) and the predicted class is a decimal number, not a class...